# DAGs - a generalization of the alignment problem
##Specification
- Name: problem15 in Rosalind
- Name your notebook as: DAG.ipynb
- options: none
- input: filename passed as first parameter to main
- output: a text file. ( using print ( .... file=someFileObject) is a handy way to do this after you have opened someFileObject as a text file). I find it handy to name these files by creating a string by concatenating the string named infile with ".out" ... rosalind5.txt.out ( for example).
-Rosalind Problem Name: Find the longest path in a DAG

As always, include a Inspection Intro Markdown that describes your specific algorithm at the beginning of the notebook, and another Inspection Results markdown at the end of the notebook that documents: your inspection team, the findings of the team, and your resolution of those findings.

Please submit your notebook, an example of one of the Rosalind files that you ran and passed, and the output that your program generated as a text file.

## Description
As a prelude to an upcoming assignment in profile alignments, please consider the related problem of generalized weighted-DAGs (directed acyclic graph). Rosalind has a very nice example that includes weighted edges connecting nodes. As applied to a simple pairwise alignment, nodes represent a specific assignment of a character pairing between a character in one of the sequences to either an initial insert/delete (indel), an extension of that indel, or a match/mismatch. 

Unlike classical pairwise alignment, the specific order to consider the resolution of scores in the graph is established using the topology of the graph ( topological ordering). This ordering will avoid recursion in developing the final score. This is another way of using Dynamic programming in a more generalized way. ( pp247-52 in Compeau and the detour on pp287-88).

## Hints
1) scoring of nodes that have no inputs (or possibly incomplete inputs) must be done carefully. Choose these wisely. In a pairwise alignment, these nodes would be considered boundary nodes.

2) The problem is quite clear on the starting/ending node to consider. Make sure to evaluate these properly.

3) each node can have many inputs or none. In the generalized DAG, there is no limit on the number of inputs. Consider a data structure here that allows you to represent the incoming edge along with a score that is computed by adding the weight of that incoming edge to the score of its preceding node.

4) As you write these classes, consider that these may be usable in the future.

5) sets and dictionaries require keys that are immutable. Those keys can be integers, or floats, or tuples. Think carefully about using tuples as containers-that-can-be-keys.

6) python has a container called a namedtuple. These are available in the container module. They are quite handy.

7) python has two functions called any() and all(); any() evaluates to true if any of its iterable arguments are True; all() evaluates to True if all of its iterable arguments are true. None, for example, evaluates to False. 

8) consider implementing a helper method called _argMax_. If you happen to have a dictionary of values, it returns the key whose paired-value is maximal among all values in the dictionary.

9) Carefully consider the implementations that are provided in the text. Specifically, the topology ordering pseudocode removes edges from the constructed DAG. This is a bit annoying, considering that we need that DAG for scoring and backtracking. What is the purpose that removing an edge is attempting to perform? Look over the Wikipedia article on "Dataflow" for an idea as to how you might avoid destroying your graph.

10) My implementation has specific objects that are nodes, edges, and the DAG itself. It is implemented in about 120 lines including whitespace.

11) Rosalind will provide test cases where the proscribed beginning and ending node have inputs/outputs that may induce cycles. In these problems, remember that we are considering the DAG that is described between (inclusively) the start and end node so the inputs to start don't matter; the outputs from end don't matter either.

 

Here is a template to use.

## Inspection Intro

ALGORITHM: 

IMPORTANT NOTE: THE TEXT FILE MUST HAVE EXACTLY ONE NEW LINE AT THE END OF THE FILE TO READ THE LAST EDGE PROPERLY
Read through the input text file taking note of the start point and end point as well as creating a list of edges.
This list of edges is called listOfUnusedEdges.

(CreateNodeDict)
Create a dictionary that has keys of the name (number) of the node and values of a list of information about that node.
The list of information includes the maximum value that has been found for that node, 
the previous node that led to that maximum value (BACKPOINTER), 
a true/false dependent on if the node is "prepared" or not (this means all edges that end at this node have been calculated),
the number of unevaluated edges that lead to the node, 
and a list of all nodes that have edges that lead to the node which starts as an empty list.

(MakeBeginAndEndChanges)
Next I made changes to the dictionary of nodes based on the beginning and end nodes from the input:
I set the beginning node's values so that it's previous value was set to false and set it to "prepared".
I then went through the dictionary of nodes and checked for nodes that aren't the start node, and have no inputs, setting
their max value to -2000 (arbitrarily low value) and set those nodes to "prepared" as well.
In the same method I have the method look through the list of all the edges that leave the end node or go into the start node
and I removed them from the list of edges

After this I iterated through the list of edges and added 1 to the number of inputs for each edge's end node

(AddInputs)
Then I added the start node from every edge to each of the end node's (of the edge) list of inputs.

Then I went through each node in the dictionary of nodes and checked to see if it's list of inputs only included nodes
whose value was -2000 and set their value to -2000 as well so they would not be used.

With all the prep work done, I then used a while loop to iterate through all of the edges that were left (viable edges) and

START WHILE LOOP: (while there are edges in the list of edge)
(PrepareEdges)
Changed all nodes whose number of unevaluated inputs were zero to "prepared"

(AddPreparedEdges)
Checked through the list of prepared edges and added edges that started at "prepared" nodes to the list of prepared edges

(EvaluatePreparedEdges)
Evaluated to see if that edge would lead to a new maximum for that edge's end node, if it did, I changed the maximum value
for that node to the new maximum. Regardless if it created a new maximum or not I removed the edge from the list of edges.
I cleared the list of prepared edges after evaluating them all.

END OF WHILE LOOP

I then printed the end node's maximum value.
Then I went backwards from the end node using the backpointers, adding each of the nodes of the path to a list.
Then I reversed the list so it was ordered correctly and printed the path with "->"s in between each node, thus printing the path.

In [19]:
''' This is an assignment I worked on as part of BME 205, a graduate level programming class where we use DP to find the longest Path in a Directed Acyclic Graph'''
# input: A start node, end node, and list of edges
# returns the path of greatest value from start node to end node and the value of the path
# Groupmates: Alan Faber, Sarah Ghasemi
# Author: Jeffrey Jacob

class DAG:
    def __init__(self, begin, end):
        ''' Constructor, saves the beginning point of the graph and the end point '''
        self.begin = begin
        self.end = end
        

    def CreateNodeDict(self, UnusedEdgeList, NodesList):
        ''' 
        Creates a dictionary of the nodes that have an edge that either comes from them or have an edge that leads to them.
        The dictionary contains a list for every node that has information on each node.
        The information is: [max value, the node previous that led to the maximum value, if the node is prepared, and number of unused edges that lead to the node]
        '''
        NodeDict = {}
        for edge in UnusedEdgeList:
            # The values 0, 0, False, 0 are placeholders for maxValue, backtracking pointer, preparedness, # of inputs, and list of inputs
            if edge[0] not in NodesList:
                NodesList.append(edge[0])
                NodeDict[edge[0]] = [0, 0, False, 0, []]
            if edge[1] not in NodesList:
                NodesList.append(edge[1])
                NodeDict[edge[1]] = [0, 0, False, 0, []]
        return NodeDict

    def MakeBeginAndEndChanges(self, NodeDict, NodesList, UnusedEdgesList, begin, end):
        '''
        Modifies the NodeDict: Changes the beginning node to prepared.
        Removes edges that lead to the begin node and edges that leave the end node.
        '''
        # set beginning node's values, remove node from Unused list, and add node to Used list
        NodeDict[begin][0] = 0
        NodeDict[begin][1] = False
        NodeDict[begin][2] = True
        NodesList.remove(begin)
        # Check for nodes that aren't the start node and have no inputs. Set their max value to -2000 so they aren't used.
        for node in NodesList:
            flag = False
            for edge in UnusedEdgesList:
                if edge[1] == node:
                    flag = True
            if flag == False:
                NodeDict[node][0] = -2000
                NodeDict[node][2] = True

        # Go through all edges that leave the end node or go into the begin node and remove them from the list
        listOfUnuseableEdges = []
        for edge in UnusedEdgesList:
            if edge[0] == end:
                listOfUnuseableEdges.append(edge)
            if edge[1] == begin:
                listOfUnuseableEdges.append(edge)
        for edge in listOfUnuseableEdges:
            UnusedEdgesList.remove(edge)

    def AddPreparedEdges(self, UnusedEdgesList, NodeDict, PreparedEdgesList):
        ''' Add edges that have not been used and whose start point is prepared '''
        for edge in UnusedEdgesList:
            if NodeDict[edge[0]][2] == True:
                PreparedEdgesList.append(edge)

    def EvaluatePreparedEdges(self, PreparedEdgesList, NodeDict, UnusedEdgesList, UsedEdgesList):
        '''
        Evaluate each prepared edge to see if it leads to a higher maxVal for the end node, update the DictOfNodes if it
        does so with the new maxVal, the new backpointer, and number of inputs not evaluated. Remove edge from the list
        of unused edges and add it to the list of used edges. Clear the list of prepared edges afterwards.
        '''
        for edge in PreparedEdgesList:
            if (edge[2] + NodeDict[edge[0]][0]) > NodeDict[edge[1]][0]:
                NodeDict[edge[1]][0] = edge[2] + NodeDict[edge[0]][0]
                NodeDict[edge[1]][1] = edge[0]
            NodeDict[edge[1]][3] -= 1
            UnusedEdgesList.remove(edge)
            UsedEdgesList.append(edge)
        PreparedEdgesList.clear()

    def PrepareEdges(self, NodeDict):
        ''' Go through the list of edges checking for nodes that have 0 inputs that are unused and set the nodes to ready '''
        for node in NodeDict:
            # if node has 0 incoming unevaluated edges
            if NodeDict[node][3] == 0:
                NodeDict[node][2] = True

    def AddInputs(self, NodeDict, UnusedEdgesList):
        ''' Adds the possible inputs to the nodes '''
        for edge in UnusedEdgesList:
            NodeDict[edge[1]][4].append(edge[0])


def main(inFile = None):
    '''
    
    reads a file and parses ut, applies methods from the dag class to find the longest weighted path
    
    '''
    listOfNodes = []
    listOfUsedEdges = []
    listOfPreparedEdges = []
    listOfUnusedEdges = []
    
    with open(inFile) as fh:
        begin = int(fh.readline().rstrip())
        end = int(fh.readline().rstrip())
        thisDAG = DAG(begin, end)
        for line in fh:
            startNode =  0
            endNode = 0
            value = 0
            temp = ""
            # Requires a newline character on the last line to work for the last edge (txt file must have a blank line at the end)
            for char in line:
                if char == "-":
                    startNode = int(temp)
                    temp = ""
                    continue
                if char == ">":
                    continue
                if char == ":":
                    endNode = int(temp)
                    temp = ""
                    continue
                if char == '\n':
                    value = int(temp)
                    continue
                temp = temp + str(char)
            tempTup = (startNode, endNode, value)
            listOfUnusedEdges.append(tempTup)
        
        # Create the dictionary of nodes with the initial values
        dictOfNodes = thisDAG.CreateNodeDict(listOfUnusedEdges, listOfNodes)
        
        # Check for nodes that aren't the start point and have no inputs, if there are any, sets their max value to -2000     
        # Go through all edges that leave the end node or go into the start node and remove them from the list
        thisDAG.MakeBeginAndEndChanges(dictOfNodes, listOfNodes, listOfUnusedEdges, begin, end)
        
        
        # Go through all edges and add 1 to a Node's number of inputs to it in the DictOfNodes
        for edge in listOfUnusedEdges:
            dictOfNodes[edge[1]][3] += 1
            
        # Add all inputs to the list of inputs for each node in the dictionary of nodes
        thisDAG.AddInputs(dictOfNodes, listOfUnusedEdges)
        
        # If a node's edges only come from a node with a -2000 value, set node's value to -2000
        for node in dictOfNodes:
            flagg = False
            for inputs in dictOfNodes[node][4]:
                if dictOfNodes[inputs][0] > -2000:
                    flagg = True
            if node == begin:
                flagg = True
            if flagg == False:
                dictOfNodes[node][0] = -2000
        
        # iterate until all edges have been evaluated
        while listOfUnusedEdges:
            # Go through the list of edges checking for nodes that have 0 inputs that are unused and set the nodes to ready
            thisDAG.PrepareEdges(dictOfNodes)
            # Add edges that have not been used and whose start point is prepared
            thisDAG.AddPreparedEdges(listOfUnusedEdges, dictOfNodes, listOfPreparedEdges)
            # evaluate each prepared edge to see if it leads to a higher maxVal for the end node, update the DictOfNodes if it
            # does so with the new maxVal, the new backpointer, and number of inputs not evaluated. Remove edge from the list
            
            # of unused edges and add it to the list of used edges. 
            thisDAG.EvaluatePreparedEdges(listOfPreparedEdges, dictOfNodes, listOfUnusedEdges, listOfUsedEdges)
        
        
        print(dictOfNodes[end][0])
        
        
        
        path = []
        currentNode = end
        # get the path backwards and put in in a list
        while currentNode is not False:
            path.append(currentNode)
            currentNode = dictOfNodes[currentNode][1]
        path = path[::-1]
        
        # print out the path
        for x in range(len(path) - 1):
            print(str(path[x]) + "->", end="")
        print(path[len(path) - 1])
        
    

    
if __name__ == "__main__":
    main(inFile = 'rosalind_ba5d.txt') 

54
14->17->28


## Inspection Results

In [ ]:
Inspection: 
    File runs correctly and is well documented, the main function is longer than it needs to be however
    abstracting more to the classes could be more simple.

Inspection: Jeffrey Jacob
    Program produces the correct output and runs quickly, code is
    well documented, some of the variables could be named better in order for
    the code to be more readable, but the extensive comments take care of that.
    program utilizes oop and proper coding conventions to count edges and create the path.

Inspection: 